<a href="https://colab.research.google.com/github/Impishmouse/StyleGAN2_model_samples/blob/main/models_generate_sample_grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate grid 3x3 samples how .pkl trained


In [ ]:
#@title Підготовка бібліотек та методів
!git clone https://github.com/Impishmouse/stylegan2.git
%cd stylegan2
%tensorflow_version 1.x
!rm -rf /content/sample_data
!pip install typer

import dnnlib
import dnnlib.tflib as tflib
from dnnlib.tflib import tfutil
from dnnlib.tflib.autosummary import autosummary
from IPython.display import Image 
import numpy as np
import math
from math import ceil
import pickle
from pathlib import Path
import pretrained_networks
import PIL.Image
from training import dataset
from training import misc
import typer
from typing import Optional
from tqdm import tqdm
import tensorflow as tf
import sys, getopt, os
from google.colab import files


def createImageGrid(images, scale=0.25, rows=1):
   w,h = images[0].size
   w = int(w*scale)
   h = int(h*scale)
   height = rows*h
   cols = ceil(len(images) / rows)
   width = cols*w
   canvas = PIL.Image.new('RGBA', (width,height), 'white')
   for i,img in enumerate(images):
     img = img.resize((w,h), PIL.Image.ANTIALIAS)
     canvas.paste(img, (w*(i % cols), h*(i // cols))) 
   return canvas

def generate_zs_from_seeds(seeds):
    zs = []
    for seed_idx, seed in enumerate(seeds):
        rnd = np.random.RandomState(seed)
        z = rnd.randn(1, *Gs.input_shape[1:]) # [minibatch, component]
        zs.append(z)
    return zs

# Trunctation psi value needed for the truncation trick
def generate_images(zs, truncation_psi):
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    if not isinstance(truncation_psi, list):
        truncation_psi = [truncation_psi] * len(zs)
        
    imgs = []
    for z_idx, z in tqdm(enumerate(zs)):
        Gs_kwargs.truncation_psi = truncation_psi[z_idx]
        noise_rnd = np.random.RandomState(1) # fix noise
        tflib.set_vars({var: noise_rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        images = Gs.run(z, None, **Gs_kwargs) # [minibatch, height, width, channel]
        imgs.append(PIL.Image.fromarray(images[0], 'RGB'))
        
    # Return array of PIL.Image
    return imgs

def generate_images_from_seeds(seeds, truncation_psi):
    return generate_images(generate_zs_from_seeds(seeds), truncation_psi)


# TODO  later remove connect to drive /  use drive links to download model
# mound google drive with models
from google.colab import drive
drive.mount('/content/drive')

# Upload unzip and set default networks
if os.path.isdir ('/content/stylegan2/trainedNetworks') == False:
  !mkdir /content/stylegan2/trainedNetworks
!gdown --id 1vQM_9_-x90t_v5hokf45LTxPD19u0nv0
!unzip -u /content/stylegan2/default_networks.zip -d /content/stylegan2/trainedNetworks/
!rm /content/stylegan2/default_networks.zip

print("Finish!")


In [ ]:
#@title Обрерить назву мережі з якої будемо робити приклади
create_one = True #@param {type:"boolean"}
net_name = "toony_b_paint_64_1024" #@param ["ukiyoe_256", "ffhq_e_256", "cat_f_256", "panda_256", "anime_512", "CartoonsAligned_1_512", "fursona_512","toony_b_paint_64_1024", "ffhq_e_104", "metFaces_1024", "wiki_art_1_1024", "wiki_art_2_1024", "comics_monsters_1024", "baby_1024", "aBeauty_1024", "asian_1024", "ffhq_f_1024", "ffhq_b_cartoon_64_1024", "CartoonsAligned_36v2_1024", "CartoonsAligned_84v3_1024", "ChSt_1_1024", "ada_aquapaint_flowers", "ada_friends_hq", "WoW_Faces_1024", "Muppets_1024", "baby_dolls_1024", "pokemon_1024", "ponies_1024", "FreaGAN_1024"]

download_zip = False #@param {type:"boolean"}

# TODO  later remove connect to drive /  use drive links to download model
networks_urls = {
    'ukiyoe_256':'/content/drive/My Drive/projects/StyleGan2/Models/256/ukiyoe-256-slim-diffAug-002789.pkl',
    'ffhq_e_256':'/content/drive/My Drive/projects/StyleGan2/Models/256/ffhq-256-config-e-003810.pkl',
    'cat_f_256':'/content/drive/My Drive/projects/StyleGan2/Models/256/stylegan2-cat-config-f.pkl',
    'panda_256':'/content/drive/My Drive/projects/StyleGan2/Models/256/stylegan2-100-shot-panda.pkl',
    'anime_512': '/content/drive/My Drive/projects/StyleGan2/Models/512/2020-01-11-skylion-stylegan2-animeportraits-networksnapshot-024664.pkl',
    'CartoonsAligned_1_512':'/content/drive/My Drive/projects/StyleGan2/Models/512/FFHQ512-CartoonsAlignedHQ31v3.pkl',
    'fursona_512':'/content/drive/My Drive/projects/StyleGan2/Models/512/network-e621-r-512-3194880.pkl',
    'toony_b_paint_64_1024':'/content/drive/My Drive/projects/StyleGan2/Models/1024/toony_blended_64_Paint_1024.pkl',
    'ffhq_e_104':'/content/drive/My Drive/projects/StyleGan2/Models/1024/stylegan2-ffhq-config-e.pkl',
    'metFaces_1024':'/content/drive/My Drive/projects/StyleGan2/Models/1024/network-snapshot-metfaces2.pkl',
    'wiki_art_1_1024':'/content/drive/My Drive/projects/StyleGan2/Models/1024/wiki_art_1_1024.pkl',
    'wiki_art_2_1024':'/content/drive/My Drive/projects/StyleGan2/Models/1024/wiki_art_2_1024.pkl',
    'comics_monsters_1024':'/content/drive/My Drive/projects/StyleGan2/Models/1024/drwho-comics-and-monsters-000042.pkl',
    'baby_1024':'/content/drive/My Drive/projects/StyleGan2/Models/1024/baby-stylegan2-config-f.pkl',
    'aBeauty_1024':'/content/drive/My Drive/projects/StyleGan2/Models/1024/asian_beauty-stylegan2-config-f.pkl',
    'asian_1024':'/content/drive/My Drive/projects/StyleGan2/Models/1024/asian-stylegan2-config-f.pkl',
    'ffhq_f_1024':'/content/stylegan2/trainedNetworks/stylegan2-ffhq-config-f.pkl',
    'ffhq_b_cartoon_64_1024':'/content/drive/My Drive/projects/StyleGan2/Models/1024/toonify/ffhq-cartoon-blended-64.pkl',
    'CartoonsAligned_36v2_1024':'/content/drive/My Drive/projects/StyleGan2/Models/1024/toonify/FFHQ-CartoonsAlignedHQ36v2.pkl',
    'CartoonsAligned_84v3_1024':'/content/drive/My Drive/projects/StyleGan2/Models/1024/toonify/FFHQ-CartoonsAlignedHQ84v3.pkl',
    'ChSt_1_1024':'/content/drive/My Drive/projects/StyleGan2/Models/1024/CS-snapshot-010327.pkl',
    'WoW_Faces_1024':'/content/drive/My Drive/projects/StyleGan2/Models/1024/ffhq-WoW-Faces-000048.pkl',
    'Muppets_1024':'/content/drive/My Drive/projects/StyleGan2/Models/1024/FFHQ-Muppets-000132.pkl',
    'baby_dolls_1024':'/content/drive/My Drive/projects/StyleGan2/Models/1024/ffhq-a312863063-generator_baby-dolls-000024.pkl',
    'pokemon_1024':'/content/drive/My Drive/projects/StyleGan2/Models/1024/ffhq-pokemon-000052.pkl',
    'ponies_1024':'/content/drive/My Drive/projects/StyleGan2/Models/1024/network-ponies-1024-151552.pkl',
    'FreaGAN_1024':'https://drive.google.com/file/d/1umM1i6x-zxgS0Psr_6hpEbqIy527r9jv/view?usp=sharing'}



def generate_grid_and_Sample (name_of_net):
  if os.path.isdir ('/content/stylegan2/results/') == False:
    !mkdir /content/stylegan2/results/

  result_dir = (f"/content/stylegan2/results/{name_of_net}/")
  if os.path.isdir(result_dir) == False:
    !mkdir $result_dir

  seeds = [1,2,3,4,5,6,7,8,9] 
  result_scale = 0.25
  zs = generate_zs_from_seeds(seeds)
  imgs = generate_images(zs, 1.0)
  w,h = imgs[0].size
  canvas = PIL.Image.new('RGBA', (w,h), 'white')
  canvas.paste(imgs[0])

  grid = createImageGrid(imgs, scale=result_scale, rows=3)
  imW, imH = grid.size
  real_size = (imW / 3)/result_scale

  canvas.save (result_dir+(f"sample.png"))
  grid.save (result_dir+"grid.png")
  print (f"Sample for {name_of_net} Done!")

def run_noise_loop(pkl_path, truncation_level, output_dir):
  temp_outputs = Path('interpolation')
  temp_movie = temp_outputs.with_suffix(".mp4")
  net_path_src = Path(pkl_path)
  !python grid_vid.py str(pkl_path) \
  --truncation-psi 0.75 \
  --grid-size 3 3 --duration-sec 10 --smoothing-sec 1 \
  --output-width 768 --mp4 "/content/stylegan2/results/toony_b_paint_64_1024/"+$temp_movie


if (create_one == False):
 !rm -rf /content/stylegan2/results/
 !mkdir /content/stylegan2/results/  
 for one_net_url in networks_urls:
  pkl_url = networks_urls[one_net_url]
  _G, _D, Gs = pretrained_networks.load_networks(pkl_url)
  noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
  generate_grid_and_Sample (one_net_url) 
else:
  pkl_url = networks_urls[net_name]
  _G, _D, Gs = pretrained_networks.load_networks(pkl_url)
  noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
  generate_grid_and_Sample (net_name)
  result_dir = (f"/content/stylegan2/results/{net_name}/")
  if os.path.isdir(result_dir) == False:
    !mkdir $result_dir

  run_noise_loop(pkl_url, 0.75, result_dir)



if download_zip == True:
  !rm -rf /content/stylegan2/model_samples.zip
  !zip -r model_samples.zip /content/stylegan2/results
  files.download("/content/stylegan2/model_samples.zip")

9it [00:01,  6.00it/s]


Sample for toony_b_paint_64_1024 Done!
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `python grid_vid.py str(pkl_path)   --truncation-psi 0.75   --grid-size 3 3 --duration-sec 10 --smoothing-sec 1   --output-width 768 --mp4 "/content/stylegan2/results/toony_b_paint_64_1024/"+interpolation.mp4'


In [ ]:
!python grid_vid.py --help

Usage: grid_vid.py [OPTIONS] NET

Arguments:
  NET  [required]

Options:
  --mp4 PATH                      [default:
                                  output.mp4]

  --truncation-psi FLOAT          [default: 0.5]
  --grid-size <INTEGER INTEGER>...
                                  [default: 1, 1]
  --duration-sec FLOAT            [default: 60.0]
  --smoothing-sec FLOAT           [default: 1.0]
  --mp4-fps INTEGER               [default: 30]
  --mp4-codec TEXT                [default:
                                  libx264]

  --random-seed INTEGER           [default: 1000]
  --minibatch-size INTEGER        [default: 8]
  --output-width INTEGER
  --install-completion [bash|zsh|fish|powershell|pwsh]
                                  Install
                                  completion for
                                  the specified
                                  shell.

  --show-completion [bash|zsh|fish|powershell|pwsh]
                                  Show completion
       

In [ ]:
temp_outputs = Path('interpolation')
temp_movie = temp_outputs.with_suffix(".mp4")

!python grid_vid.py /content/drive/My\ Drive/projects/StyleGan2/Models/1024/toony_blended_64_Paint_1024.pkl \
 --truncation-psi 0.75 \
 --grid-size 3 3 --duration-sec 10 --smoothing-sec 1 \
 --output-width 768 --mp4 "/content/stylegan2/results/toony_b_paint_64_1024/"+$temp_movie


    #cmd = ['python', 'grid_vid.py', 
     #       working_dir + '/' + str(pkl_path),
      #      '--truncation-psi', truncation, 
       #     '--grid-size', '3', '3', 
        #    '--duration-sec', '10', 
         #   '--smoothing-sec', '1',
          #  '--output-width', str(3*256),
           # '--mp4', working_dir + '/' + str(output_dir),]

Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Generating latent vectors...
2020-11-06 23:12:13.340889: W tensorflow/core/common_runtime/bfc_allocator.cc:305] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.
2020-11-06 23:12:17.895560: W tensorflow/core/common_runtime/bfc_allocator.cc:239] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.27GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be perfor

In [ ]:
# check folder space. 
!du  -sh /content/drive/My\ Drive/projects/StyleGan2